In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import text_to_word_sequence

In [1]:
path= "../raw_data/train_df_ml_clean.csv"

In [ ]:
clean_df = pd.read_csv(path)

1288134

In [5]:
small_clean_df = clean_df.sample(frac=0.1)
len(small_clean_df)

128813

In [6]:
small_clean_df.head()

,label,text,clean_text
22387,1,Read if only for the Rooster: Again David Seda...,read if only for the rooster again david sedar...
419673,1,Glass: An Artist's Medium: I liked this book v...,glass an artist medium i liked this book very ...
866848,1,The power of the Three Wolf Moon Shirt continu...,the power of the three wolf moon shirt continu...
394378,1,A Laurel for the Master: It's often said Chand...,a laurel for the master it often said chandler...
586389,1,"Batteries, yeah!: Got them in a timely manner....",battery yeah got them in a timely manner neede...


In [12]:
X_small = small_clean_df['clean_text'].to_numpy()
y_small = small_clean_df['label'].to_numpy()

X_train_small, X_val_small, y_train_small, y_val_small = train_test_split(
    X_small, y_small, test_size=0.2, random_state=42
)

array(['doesnt work doesnt work tried it on three different tv and got weerd line with all resolution tried interlaced and progressive scan',
       'i still know who wa the best actor will smith before this movie willy wa a good actor but now he is the whole best actor why watch this movie youll recognize',
       'good mix of hip hoprb i first fell in love with olivia after her single bizounce came out now im a huge fan ever since her album droppedgreat voice great beat and great lyric are what make this album soooo goodif your a fan of hip hop and rb then i really suggest buying the album ya wont be dissapointed',
       ...,
       'this map get you everywhere when we first looked at this map we didnt think it would be accurate once we arrived in scotland we realized that it wa extremely accurate there are just not that many road there a we are used to in the u i would recommend this to anyone traveling in scotland',
       'this book stink i dont mean to be frank or rude but i don